## 1. Chipotle locations from Thinknum
<p>Some of us have thought about opening a business and most of us have thought about grabbing a burrito from Chipotle. To do either of these things we need to know where the current Chipotle stores are (and are not) located. </p>
<p>In this notebook, the <code>leaflet</code> package in R and data from <a href="https://www.thinknum.com/">Thinknum</a> are used to find potential locations for new Chipotle restaurants as well as the closest place to grab a burrito. </p>
<p>Thinknum tracks thousands of websites capturing and indexing vast amounts of public data. Let's get started by loading and exploring data on every Chipotle restaurant tracked in the Thinknum data. Then we will build several <code>leaflet</code> maps that we can use to explore the data and to see where we might recommend opening a Chipotle.</p>
<p><img src="https://s3.amazonaws.com/assets.datacamp.com/production/project_478/img/Chipotle_Mexican_Grill_logo.png" alt="logo"></p>

In [73]:
# Load tidyverse, leaflet, and leaflet.extras
library(tidyverse)
library(leaflet)
library(leaflet.extras)
library(sf)

# Read datasets/chipotle.csv into a tibble named chipotle using read_csv
chipotle <- read_csv("datasets/chipotle.csv")

# Print out the chipotle tibble using the head function
head(chipotle)

Parsed with column specification:
cols(
  id = col_integer(),
  street = col_character(),
  city = col_character(),
  st = col_character(),
  ctry = col_character(),
  lat = col_double(),
  lon = col_double(),
  closed = col_character()
)


id,street,city,st,ctry,lat,lon,closed
1358023,"121 N. La Cienega Blvd.,",Los Angeles,NA,United States,34.07366,-118.37650,t
1358955,"24369 Cedar Rd,",Lyndhurst,OH,United States,41.50338,-81.50298,t
1359012,"1130 West Grove Ave.,",Mesa,NA,United States,33.39023,-111.85550,t
1359490,"6316 Delmar,",St. Louis,MO,United States,38.65559,-90.30255,t
1359574,"1464 St. Louis Galleria,",St. Louis,MO,United States,38.63275,-90.34855,t
1359575,"8301 Westchester,",Dallas,TX,United States,32.86364,-96.80657,t


## 2. Do Chipotle stores ever close?
<p>First, let's make sure we don't recommend opening a location where one has already been closed. This may also prevent many a disappointing Chipotle run to closed locations. Rather than looking just at the city/state pairs in the data, we can plot all of the closed locations to see exactly where the restaurants were located. </p>
<p><code>leaflet</code> maps work with the <code>%&gt;%</code> operator so we can pipe our <code>chipotle</code> data directly into a chain of function calls to produce an interactive map. All of the Chipotle locations have already been geocoded. The <code>leaflet</code> package will scan our column names for variables that are likely lat and lon and if we use a common naming convention (e.g., lat/lon, latitude/longitude, or lat/lng), <code>leaflet</code> will automatically know which columns contain our coordinates. </p>
<p>Because of their interactive features <code>leaflet</code> maps can be especially helpful for exploratory data analyses. After we make our first map take a minute or two to zoom and pan the map to explore where Chipotle locations have closed. </p>

In [75]:
# Create a leaflet map of all closed Chipotle stores
closed_chipotles <- 
chipotle %>% 
  # Filter the chipotle tibble to stores with a value of t for closed
  filter(closed == 't') %>% 
  leaflet() %>% 
  # Use addTiles to plot the closed stores on the default Open Street Map tile
  addTiles() %>%
  # Plot the closed stores using addCircles
  addCircles() 

# Print map of closed chipotles
closed_chipotles

Assuming "lon" and "lat" are longitude and latitude, respectively


HTML widgets cannot be represented in plain text (need html)

## 3. How many Chipotle stores have closed?
<p>After exploring the map, the first question that comes to mind is why did these particular locations close? In fact, why would any Chipotle ever close? Unfortunately, questions like this defy logic, so after quickly counting up the closed locations, this notebook moves on to the more important question of "Where should the next Chipotle be opened?". </p>
<p>Rather than counting up all of the circles on our interactive map, we can use <code>dplyr</code> to quickly count the number of closed stores. After we note this, we'll create a new <code>tibble</code> that removes the closed locations from our data so that we do not confuse them for open locations in future maps. </p>

In [77]:
# Use count from dplyr to count the values for the closed variable
chipotle %>% 
  count(closed == 't')

# Create a new tibble named chipotle_open that contains only open chipotle 
chipotle_open <-
  chipotle %>% 
  filter(closed == "f") %>% 
  # Drop the closed column from chipotle_open
  select(-closed)

"closed == ""t""",n
FALSE,2469
TRUE,15


## 4. Where are (and aren't) there Chipotles?
<p>Where's the closest Chipotle? Perhaps, more interesting is a slightly different question, where aren't there Chipotles (in the US)? 
By mapping all of the Chipotle locations on an interactive <code>leaflet</code> map we can start to explore patterns in the geographic distribution of the chain's locations. </p>
<p>Since there are thousands of store locations, many of which are clustered closely together, we will start with a heatmap. Heatmaps are a popular option for mapping large amounts of points because they leverage a color scheme to represent the data rather than plotting each point individually. This enables users to quickly identify variation in the density of points and prevents tightly clustered points from overlapping. </p>
<p>The <code>addHeatmap</code> function comes from the <code>leaflet.extras</code> package, which contains many useful functions that extend the <code>leaflet</code> package.</p>
<p>Zooming and panning the map the heatmap will adjust based on the current view of the map. </p>
<p>Are there any Chipotle deserts in the United States? </p>

In [79]:
# Pipe chipotle_open into a chain of leaflet functions
chipotle_heatmap <- 
chipotle_open %>% 
  leaflet() %>% 
  # Use addProviderTiles to add the CartoDB provider tile 
  addProviderTiles("CartoDB") %>%
  # Use addHeatmap with a radius of 8
  addHeatmap(radius = 8)

# Print heatmap
chipotle_heatmap

Assuming "lon" and "lat" are longitude and latitude, respectively


HTML widgets cannot be represented in plain text (need html)

## 5. Which States have the fewest Chipotles?
<p>Using the greyscale <code>CartoDB</code> provider tile with a colorful heatmap palette quickly revealed both the presence and absence of Chipotle stores throughout the United States. Using a greyscale base map is often useful for exploratory data analysis as it  makes patterns of Chipotle clusters and Chipotle deserts clearly stand out on the map. </p>
<p>For example, panning and zooming the map reveals that Chipotles are often located on horizontal or vertical lines. Zooming in further reveals that this is because stores are often located near interstate highways (check out Utah for an example). </p>
<p>Let's take a closer look at where there are not Chipotle stores by quantifying the Chipotle deserts using <code>dplyr</code> to count the number of Chipotle locations in each US state. </p>

In [81]:
# Create a new tibble called chipotles_by_state to store the results
chipotles_by_state <- 
chipotle_open %>% 
  # Filter the data to only Chipotles in the United States
  filter(ctry == "United States") %>% 
  # Count the number of stores in chipotle_open by st
  count(st) %>% 
  # Arrange the number of stores by state in ascending order
  arrange(n)

# Print the state counts
chipotles_by_state

st,n
MS,1
ND,1
VT,1
WY,2
MT,3
ME,5
WV,5
AR,6
DE,6
NH,7


## 6. How many States in the US?
<p>The <code>chipotle_by_state</code> tibble had 48 rows, but there are 50 fifty states in the US. Why don't we have fifty rows? Perhaps, there are two (unfortunate) states do not have a single Chipotle. Let's take a look using a couple of handy features that are included in base R. The <code>state.abb</code> vector has fifty elements, each containing a state's abbreviation. Using this vector in combination with the <code>!</code> and <code>%in%</code> operators, we can quickly and systematically determine which states do not have a Chipotle. The <code>%in%</code> operator helps us to determine which elements of one vector are present <em>in</em> another vector, while the <code>!</code> operator allows us to accomplish the inverse (i.e., finding which elements of one vector are <em>not in</em> another). </p>

In [83]:
# Print the state.abb vector
state.abb

# Use the %in% operator to determine which states are in chipotles_by_state
state.abb %in% chipotles_by_state$st

# Use the %in% and ! operators to determine which states are not in chipotles_by_state
!state.abb %in% chipotles_by_state$st

# Create a states_wo_chipotles vector
states_wo_chipotles <- state.abb[!state.abb %in% chipotles_by_state$st]

# Print states with no Chipotles
states_wo_chipotles

[1] "AL" "AK" "AZ" "AR" "CA" "CO" "CT" "DE" "FL" "GA" "HI" "ID" "IL" "IN" "IA"
[16] "KS" "KY" "LA" "ME" "MD" "MA" "MI" "MN" "MS" "MO" "MT" "NE" "NV" "NH" "NJ"
[31] "NM" "NY" "NC" "ND" "OH" "OK" "OR" "PA" "RI" "SC" "SD" "TN" "TX" "UT" "VT"
[46] "VA" "WA" "WV" "WI" "WY"

[1]  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE FALSE  TRUE
[13]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[25]  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[37]  TRUE  TRUE  TRUE  TRUE FALSE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE  TRUE
[49]  TRUE  TRUE

[1] FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
[13] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[25] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[37] FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[49] FALSE FALSE

[1] "AK" "HI" "SD"

## 7. Where to open a Chipotle I
<p>Wait a second...have I had this wrong all along? 48 + 3 = 51 states!?! Let's take a closer look at the values in our state variable that are not in the <code>state.abb</code> vector. </p>
<pre><code class="{r} language-{r}">chipotles_by_state$st[!chipotles_by_state$st %in% state.abb]
</code></pre>
<p>Ah! Washington, D.C. surprises me once again. DC is a district and not a state, so 51 seems ok for this notebook. </p>
<p>Let's focus on the only state in the contiguous United States that does not have a Chipotle: South Dakota. If we were to open a Chipotle location in South Dakota, how might we go about selecting proposed locations? In the following chunks of code, we look at several maps to explore how the location of current Chipotles as well as geographic, transportation, and governmental features of the state may inform this decision.  </p>
<p>First, let's take a look at how South Dakota's population is distributed across the state using data from the US Census. The <code>tidycensus</code> package in R can be used to facilitate access to census data from R. For a great example, check out [this](
https://juliasilge.com/blog/using-tidycensus/) blog post by Julia Silge.  </p>

In [85]:
# Load south_dakota_pop.rds into an object called south_dakota_pop
south_dakota_pop <- readRDS("datasets/south_dakota_pop.rds")

# Create color palette to color map by county population estimate
pal <- colorNumeric(palette = "viridis", domain = south_dakota_pop$estimate)

sd_pop_map <-
  south_dakota_pop %>%
  leaflet() %>%
  addProviderTiles("CartoDB") %>%
  # Add county boundaries with addPolygons and color by population estimate
  addPolygons(stroke = FALSE, fillOpacity = 0.7, color = ~ pal(estimate)) %>%
  # Add a legend using addLegend 
  addLegend(pal = pal, values = ~estimate, title = "Population")

# Print map of South Dakota population by county
sd_pop_map

HTML widgets cannot be represented in plain text (need html)

## 8. Where to open a Chipotle II
<p>Minnehaha and Pennington counties really stand out on population map. These counties are home to Sioux Falls and Rapid City, respectively. Let's take a closer look at each of the two largest cities in South Dakota to consider what features of the base map may be important when selecting the location of a Chipotle?</p>
<p>Sioux Falls has a larger population, but Rapid City is proximate to Badlands National park, which has a million visitors a year. Additionally, we should note that I-90, a major interstate in America, runs through both cities.  </p>
<p>Let's plot a proposed Chipotle location in each city to further our exploration. The <code>geocode</code> function from the <code>ggmap</code> package can be used to geocode locations in R. Alternatively, the <code>addReverseSearchOSM</code> function from the <code>leaflet.extras</code> package can be used to enable geocoding via mouse click on an interactive map. </p>

In [87]:
# Load chipotle_sd_locations.csv that contains proposed South Dakota locations  
chipotle_sd_locations <- read_csv("datasets/chipotle_sd_locations.csv")

# limit chipotle store data to locations in states boardering South Dakota
chipotle_market_research <- 
  chipotle_open %>% 
  filter(st %in% c("IA", "MN", "MT", "ND", "NE", "WY")) %>% 
  select(city, st, lat, lon) %>% 
  mutate(status = "open") %>% 
  # bind the data on proposed SD locations onto the open store data
  bind_rows(chipotle_sd_locations) 

# print the market research data
chipotle_market_research

Parsed with column specification:
cols(
  city = col_character(),
  st = col_character(),
  lat = col_double(),
  lon = col_double(),
  status = col_character()
)


city,st,lat,lon,status
Lincoln,NE,40.81311,-96.64009,open
Brooklyn Park,MN,45.09414,-93.38321,open
Eagan,MN,44.83598,-93.15196,open
Champlin,MN,45.15832,-93.39078,open
Woodbury,MN,44.94463,-92.90468,open
Columbia Heights,MN,45.06271,-93.24830,open
Fargo,ND,46.85503,-96.86172,open
Iowa City,IA,41.65975,-91.53489,open
Minnetonka,MN,44.91803,-93.50246,open
Minneapolis,MN,44.95535,-93.29721,open


## 9. Where to open a Chipotle III
<p>Let's map our proposed Chipotle restaurants in Sioux Falls and Rapid City so we can quickly see how close they are to the nearest open location. </p>
<p>So far our maps have been built using a single base map (e.g., <code>CartoDB</code>) with a single data layer (e.g., circle markers or polygons).  The <code>leaflet</code> package works like <code>ggplot2</code> and allows for multiple data layers on the same map.  Then <code>addLayerControls</code> will enable users to have control of which layers are visible. </p>
<p>Let's apply this concept in a new <code>leaflet</code> map that plots all of the open and proposed Chipotle locations in South Dakota and its bordering states. Then adding a second layer to draw a circle around each of the proposed locations to determine if there is an open store within 100 miles. </p>
<p>When using a categorical variable to create a color palette, colors can be mapped directly to the levels of the factor (i.e., there is one color in the palette for each level of the factor) or the palette can be interpolated by the <code>colorFactor</code> function to create the necessary number of colors. </p>

In [89]:
# Create a blue and red color palette to distinguish between open and proposed stores
pal <- colorFactor(palette = c("Blue", "Red"), domain = c("open", "proposed"))

# Map the open and proposed locations
sd_proposed_map <-
  chipotle_market_research %>% 
  leaflet() %>% 
  # Add the Stamen Toner provider tile
  addProviderTiles("Stamen.Toner") %>%
  # Apply the pal color palette
  addCircles(color = ~pal(status)) %>%
  # Draw a circle with a 100 mi radius around the proposed locations
  addCircles(data = chipotle_sd_locations, radius = 100 * 1609.34, color = ~pal(status), fill = FALSE) 

# Print the map of proposed locations 
sd_proposed_map

Assuming "lon" and "lat" are longitude and latitude, respectively
Assuming "lon" and "lat" are longitude and latitude, respectively


HTML widgets cannot be represented in plain text (need html)

## 10. Where to open a Chipotle IV
<p>It looks like there is a Chipotle within a 100 miles of the proposed Sioux Falls location, but not Rapid City. This is helpful to know but perhaps even more helpful would be to understand all of the locations that are closer to a proposed Chipotle than to an open one. </p>
<p>Voronoi polygons can be used to plot a polygon around each location. The bounds of each polygon will enclose all of the points on the map that are closest to a specific Chipotle. These polygons can then be used to visualize an approximation of the area covered by each Chipotle.  </p>
<p>Where should the next Chipotle be opened (hint: there has never been a wrong answer to this question!)? </p>

In [91]:
# load the Voronoi polygon data 
polys <- readRDS("datasets/voronoi_polygons.rds")

voronoi_map <- 
  polys %>%
  leaflet() %>%
  # Use the CartoDB provider tile
  addProviderTiles(provider="CartoDB") %>%
  # Plot Voronoi polygons using addPolygons
  addPolygons(fillColor = ~pal(status), weight = 0.5, color = "black") %>%
  # Add proposed and open locations as another layer
  addCircleMarkers(data = chipotle_market_research, label = ~city, 
                   color = ~pal(status))

# Print the Voronoi map
voronoi_map

Assuming "lon" and "lat" are longitude and latitude, respectively


## 11. Where should the next Chipotle be opened?
<p>After loading thousands of Chipotle locations from Thinknum, creating tables and a heatmap, identifying the shocking fact that South Dakota does not have a single Chipotle, using exploratory data analysis to investigate where a store location may make sense, including</p>
<ul>
<li>creating a county-level choropleth map showing population,</li>
<li>drawing circles with 100-mile radii around each proposed location, and</li>
<li>mapping Voronoi polygons to estimate the area covered by each proposed Chipotle, </li>
</ul>
<p>It all comes down to this: <strong>Where should the next Chipotle be opened?</strong></p>

In [ ]:
# Where should the next Chipotle store be? 
next_chipotle <- tibble(location = c("Rapid City, SD", "Sioux Falls, SD"),
                        open_new_store = c("FALSE", "TRUE"))